# Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly_resampler import register_plotly_resampler, FigureResampler, FigureWidgetResampler
from plotly.subplots import make_subplots
from scipy.signal import find_peaks
from scipy.fft import fft, fftfreq, ifft
from scipy.stats import linregress
import matplotlib.pyplot as plt
import plotly.io as pio
pio.kaleido.scope.mathjax = None

register_plotly_resampler(mode='auto')

# Functions
Run this cell to define all functions used in this notebook

In [11]:
def csv_to_dataframe(force_csv=str, mm_csv=str):
    """ 
    Reads csv files output by the GDS software and merged them into a single dataframe.

    INPUTS:
    force_csv: str
        Path to the csv file containing the force data.

    mm_csv: str
        Path to the csv file containing the displacement data.

    OUTPUTS:
    mm_df: pd.DataFrame
        Dataframe containing the displacement and force data.
    """

    if mm_csv == None:
        mm_df = pd.read_csv(force_csv, header=0)
    
    else:
        force_df = pd.read_csv(force_csv, header=1, names=['time', 'x_force', 'y_force'], usecols=[0,1,2])
        mm_df = pd.read_csv(mm_csv, header=1, names=['time', 'x_disp', 'y_disp', 'y_motor_disp', 'x_motor_disp'], usecols=[0,1,2,3,4])

        # add x_force and y_force to the main dataframe
        mm_df['x_force'] = force_df['x_force']
        mm_df['y_force'] = force_df['y_force']

    return mm_df

def add_cols(dataframe):
    """
    Adds columns to the dataframe for analysis.

    INPUTS:
    dataframe: pd.DataFrame
        Dataframe containing the displacement and force data.

    OUTPUTS:
    dataframe: pd.DataFrame
        Dataframe containing the displacement and force data with additional columns.
    """

    # Add friction (F / x = k)
    dataframe['friction'] = -dataframe['x_force'] / dataframe['y_force']

    # Cropping data so friction is greater than 0
    dataframe = dataframe[dataframe['friction'] > 0]

    # Mean velocity
    dataframe['x_speed'] = np.gradient(dataframe['x_motor_disp'], dataframe['time'])

    # Smoothing velocity to be more step-wise
    dataframe['x_speed'] = dataframe['x_speed'].rolling(200).median()

    # Rate of change of friction with respect to displacement
    dataframe['friction_slope'] = np.gradient(dataframe['friction'], dataframe['x_motor_disp'])

    return dataframe

def find_stress_drops(dataframe, stddev_coef):
    """ 
    Finds the stress drops in the friction data.
    This is based of first-differencing the friction data and finding the peaks IN THE NEGATIVE of the data (detecting troughs).
    Peaks preceding the troughs are found by starting at the trough and looking for the maximum friction value in previous indices.

    INPUTS:
    dataframe: pd.DataFrame
        Dataframe containing the displacement, force, and friction data.

    stddev_coef: float
        Coefficient for standard deviation to filter out false picks.

    OUTPUTS:
    peaks: list
        List of indices where the peaks are located.
    
    troughs: list
        List of indices where the troughs are located.

    friction_drops: list
        List of the magnitudes of the drops in friction.
    """

    # initialize lists
    dataframe['troughs'] = False
    dataframe['peaks'] = False
    dataframe['friction_drop'] = 0

    # Loading the friction (stress drops = inverted peaks)
    friction = dataframe['friction']
    friction_slope = dataframe['friction_slope']

    """BELOW HERE IS THE STRESS DROP DETECTION ALGORITHM, BASED ON FIRST DIFFERENCE OF FRICTION"""

    # Normalizing the inverted friction
    friction_slope = (friction_slope - np.min(friction_slope))/(np.max(friction_slope) - np.min(friction_slope))
    friction = (friction - np.min(friction))/(np.max(friction) - np.min(friction))

    # Detect peaks of a certain prominence, and minimum distance from each other
    # may require adjustment for different data sets
    troughs, _ = find_peaks(-1*friction_slope, distance=20, prominence=0.1) # threshold=30, prominence=120

    # Calculate the standard dev of the friction slope data
    stddev = np.std(friction_slope)
    mean = np.mean(friction_slope)

    # Convert friction slope to numpy array
    friction_slope = friction_slope.to_numpy()
    friction = friction.to_numpy()

    # filter troughs < 'stddev_coef' std from mean, take initial length of list to calculate drop %
    pre_len = len(troughs)
    troughs = [trough for trough in troughs if abs(friction_slope[trough] - mean) > stddev_coef*stddev]

    """ABOVE HERE IS THE STRESS DROP DETECTION ALGORITHM, BASED ON FIRST DIFFERENCE OF FRICTION"""

    # For each index in troughs, check to see if there is a lower friction value in the last 15 samples (false pick)
    # Then search indices surrounding the trough pick to see if there are lower values (true drop)
    for trough in troughs:
        if trough >= 15: # Above index of 15, to solve any indexing issues. Introduces error as nothing i<15 will be affected
            if np.min(friction[trough-15:trough]) < friction[trough]: # Check for lower friction value in i-15
                troughs.remove(trough)

        # Checks friction values locally to find true stress drop trough
        if troughs != [] and trough in troughs and trough >= 3 and trough <= len(dataframe) - 4:
            # If the pick is too far forward (+ X +/-)
            # This deals with the flat case too (moved the stress drop to the left as far as possible)
            while friction[trough] - friction[trough - 1] >= 0 and friction[trough] - friction[trough + 1] <= 0:
                troughs.remove(trough) # Removing peak in peaks list
                trough = trough - 1 # Replacing peak with peak - 1
                troughs.append(trough) # Replacing peak value in peaks list (with peak - 1)
            print(trough, friction[trough], friction[trough - 1], friction[trough + 1])
            while friction[trough] - friction[trough + 1] >= 0 and trough < len(dataframe) - 2:
                troughs.remove(trough) # Removing peak in peaks list
                trough = trough + 1 # Replacing peak with peak - 1
                troughs.append(trough) # Replacing peak value in peaks list (with peak - 1)
                print(trough, friction[trough], friction[trough - 1], friction[trough + 1])
                print(friction[trough] - friction[trough + 1] >= 0 and trough < len(dataframe) - 2)

    # Iterate over each peak and look for the maximum friction value in the last 5 indices to find
    # the stress "peak" preceding the drop
    peaks = troughs.copy()

    for peak in peaks:
        if peaks!= [] and peak >= 5: # Only look at peaks above i=5, to solve any indexing issues (also check for empty list)
            while friction[peak] - friction[peak - 1] <= 0: # While the previous point's friction value is higher
                peaks.remove(peak) # Remove the current value, replace with the point before it
                peak = peak - 1
                peaks.append(peak)

    # Calculate the friction drop between each peak/trough pair
    friction = dataframe['friction'].to_numpy() # redifine so it's not normalized

    friction_drops = []

    for peak, trough in zip(peaks, troughs):    

        if peak == trough or friction[peak] - friction[trough] <= 0.01:
            peaks.remove(peak)
            troughs.remove(trough)
            continue

        else:
            friction_drops.append(friction[peak] - friction[trough])

    # print dropped # of peaks
    post_len = len(troughs)
    drop_count = pre_len - post_len
    print(f'Dopped {drop_count} picks')

    """ For debugging """
    # plt.plot(np.arange(0, len(friction_slope)), friction_slope)
    # plt.plot(np.arange(0, len(friction)), friction)
    # plt.scatter(peaks, friction[peaks])
    # plt.axhline(stddev_coef*stddev + mean)
    # plt.axhline(-stddev_coef*stddev + mean)
    # plt.show()

    # dataframe['peaks'].iloc[peaks] = True

    # stress_drops = dataframe.loc[force['peaks'] == True]

    return peaks, troughs, friction_drops

def add_stress_drops(dataframe, window_size, stddev_coef):
    """ 
    Adds stress drops to the dataframe using the find_stress_drops function.
    The magnitude of the friction drop is added to the same rows as the troughs.

    INPUTS:
    dataframe: pd.DataFrame
        Dataframe containing the displacement, force, and friction data.
    
    window_size: int
        Size of the window to divide the dataframe into for analysis.

    stddev_coef: float
        Coefficient for standard deviation to filter out false picks.

    OUTPUTS:
    dataframe: pd.DataFrame
        Dataframe containing the displacement, force, and friction data with additional columns for stress drops.
    """

    # List of True/False values that will be turned into a df (force) column, where 
    peak_list = []
    trough_list = []
    friction_drop_list = []

    # Dividing the df into chunks of window_size
    for chunk in np.array_split(dataframe, len(dataframe) // window_size + 1):
        
        # Chunk-length list of Falses
        peak_temp_list = np.full(len(chunk), False)
        trough_temp_list = np.full(len(chunk), False)
        friction_drop_temp_list = list(np.full(len(chunk), 0))
        
        # Find peaks
        peaks, troughs, friction_drops = find_stress_drops(chunk, window_size, stddev_coef)

        # Replace False temp_lists with True where peaks are
        peak_temp_list[peaks] = True
        trough_temp_list[troughs] = True

        for trough, friction_drop in zip(troughs, friction_drops):
           friction_drop_temp_list[trough] = friction_drop

        # Adding the temp_lists to the main _lists
        peak_list = peak_list + list(peak_temp_list)
        trough_list = trough_list + list(trough_temp_list)
        friction_drop_list = friction_drop_list + list(friction_drop_temp_list)
        
        print(f'{len(trough_list)} / {len(dataframe)}')

    dataframe['peaks'] = peak_list
    dataframe['troughs'] = trough_list
    dataframe['friction_drops'] = friction_drop_list

    # stress_drops = force.loc[force['troughs'] == True]

    return dataframe

def save_csv(dataframe, name):
    """
    Saves the dataframe to a csv file.

    INPUTS:
    dataframe: pd.DataFrame
        Dataframe to be saved.
    
    name: str
        Name of the csv file to be saved.
    """
    
    dataframe.to_csv(name, index=False)

def clean_data(dataframe, bound_variable, upper_bound, lower_bound, sort_variable):
    """
    Removes data points that are outside of the bounds specified.

    INPUTS:
    dataframe: pd.DataFrame
        Dataframe to be cleaned.

    variable: str
        Column in the dataframe to be cleaned.

    upper_bound: float
        Upper bound of the data to be kept.

    lower_bound: float
        Lower bound of the data to be kept.
    """

    dataframe = dataframe[(dataframe[bound_variable] < upper_bound) & (dataframe[bound_variable] > lower_bound)]

    dataframe = dataframe.sort_values(by=sort_variable)

    return dataframe

In [12]:
hemisphere_20 = csv_to_dataframe('C:/Users/stickslip/Documents/leki-files/mcgill-natural-faults/surfaces/bumpy_blocks/newbatch1/20spherevscement-2024-06-19-kn.csv',
                                 'C:/Users/stickslip/Documents/leki-files/mcgill-natural-faults/surfaces/bumpy_blocks/newbatch1/20spherevscement-2024-06-19-mm.csv')

hemisphere_20 = add_cols(hemisphere_20)

hemisphere_20 = clean_data(hemisphere_20, 'friction', 10, 0, 'x_motor_disp')

C:\Users\stickslip\AppData\Local\Temp\ipykernel_22888\3284219482.py:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\stickslip\AppData\Local\Temp\ipykernel_22888\3284219482.py:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\stickslip\Documents\leki-files\mcgill-natural-faults\lab-pc-env\Lib\site-packages\numpy\lib\function_base.py:1242: RuntimeWarning:

divide by zero encountered in divide

c:\Users\stickslip\Documents\leki-files\mcgill-natural-faults\lab-pc-env\Lib\site-packages\numpy\

In [21]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=hemisphere_20['x_motor_disp'], y=hemisphere_20['friction'], mode='lines', name='20 Spheres'), secondary_y=False)
fig.add_trace(go.Scatter(x=hemisphere_20['x_motor_disp'], y=hemisphere_20['x_speed']*1000, mode='lines', name='20 Spheres x-speed'), secondary_y=True)
fig.add_trace(go.Scatter(x=hemisphere_20['x_motor_disp'], y=hemisphere_20['y_force'], mode='lines', name='20 Spheres y-force'), secondary_y=True)



fig.update_layout(width=1100, 
                  height=600, 
                  title='Friction vs. Displacement', 
                  xaxis_title='Displacement (mm)', 
                  yaxis_title='Friction Coefficient', 
                  font=dict(size=16),
                  xaxis=dict(showgrid=True, zeroline=True),
                  yaxis=dict(showgrid=True, zeroline=True, range=[0,1]))# ,
                  # paper_bgcolor="rgba(0, 0, 0, 0)",
                  # plot_bgcolor="rgba(0, 0, 0, 0)")
fig.update_yaxes(title_text="Force (kN) / Speed (um/s)", secondary_y=True)


FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'tyle="color:#fc9944">~0.02</i>'),
              'type': 'scatter',
              'uid': 'af49d7cc-4c84-41fd-b168-115fd75f3eef',
              'x': array([-14.9999978 , -14.99792654, -14.98114783, ...,   1.00646878,
                            1.03465014,   1.03610323]),
              'xaxis': 'x',
              'y': array([1.28133904e-04, 2.17844418e-03, 4.40844559e-06, ..., 1.86728458e-01,
                          1.86142100e-01, 1.79160478e-01]),
              'yaxis': 'y'},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'tyle="color:#fc9944">~0.02</i>'),
              'type': 'scatter',
              'uid': '47fe53f7-a787-4672-95d7-70c027381f67',
              'x': array([-14.9999978 , -14.99517914, -14.97168839, ...,   1.00075011,
                            1.0304501 ,   1.03610323]),
              'xaxis': 'x',
          